In [ ]:
# TODO:
# Get data for Lower 48. See if another language will help us get data faster
# Put make_buffer, weather api calls and folium map in python module
# See if we can use mapclassify to color the pins


In [9]:
import folium
from folium.plugins import MarkerCluster
import numpy as np


# Bounding box around area of interest. Do entire lower 48 when you have time to run it. 
# Faster language might be ideal for this part? Timer?
west = -108.5 #-114.7
east = -104.5 #-102.3
north = 41#45.1
south = 37#33.1

# Resolution of grid of points
num_lat_points = 10
num_lon_points = 10

# Generate latitude and longitude points
latitudes = np.linspace(south, north, num_lat_points)
longitudes = np.linspace(west, east, num_lon_points)

m = folium.Map(location=[(north+south)/2,(east+west)/2], zoom_start=6)
cluster = MarkerCluster().add_to(m)

# Add generated coordinate pairs to cluster layer
for lat in latitudes:
    for lon in longitudes:
        folium.Marker([lat, lon]).add_to(cluster)

m

In [10]:
import geopandas as gpd
import requests
from shapely.geometry import Point

In [11]:


num_requests = 0

office_data = {
    'geometry' : [],
    'grid_x' : [],
    'grid_y' : [],
    'grid_id' : [],
    'original_point' : []
}
for lat in latitudes:
    for lng in longitudes:
        num_requests += 1
        print(f"{num_requests} requests made")
        response = requests.get(f'https://api.weather.gov/points/{lat},{lng}')
        if response.status_code != 200: print("unable to retrieve observation data")
        else:
            response_json = response.json()
            properties = response_json['properties']
            office_data['geometry'].append(Point(properties['relativeLocation']['geometry']['coordinates']))
            office_data['grid_x'].append(properties['gridX'])
            office_data['grid_y'].append(properties['gridY'])
            office_data['grid_id'].append(properties['gridId'])
            office_data['original_point'].append((lat,lng))
                                       
        
        

1 requests made
2 requests made
3 requests made
4 requests made
5 requests made
6 requests made
7 requests made
8 requests made
9 requests made
10 requests made
11 requests made
12 requests made
13 requests made
14 requests made
15 requests made
16 requests made
17 requests made
18 requests made
19 requests made
20 requests made
21 requests made
22 requests made
23 requests made
24 requests made
25 requests made
26 requests made
27 requests made
28 requests made
29 requests made
30 requests made
31 requests made
32 requests made
33 requests made
34 requests made
unable to retrieve observation data
35 requests made
36 requests made
37 requests made
38 requests made
39 requests made
40 requests made
41 requests made
42 requests made
43 requests made
44 requests made
45 requests made
46 requests made
47 requests made
48 requests made
49 requests made
50 requests made
51 requests made
52 requests made
53 requests made
54 requests made
55 requests made
56 requests made
57 requests made
58 r

In [12]:
gdf_raw = gpd.GeoDataFrame(data=office_data)
gdf_raw.to_csv("colorado_points_to_grid_test.csv")

In [13]:
gdf_dropna = gdf_raw.dropna(subset=['grid_id', 'grid_x', 'grid_y'])
gdf_clean = gdf_dropna.drop_duplicates()
len(gdf_clean)

98

In [14]:
from pyproj import Transformer, CRS
from shapely.geometry import Point
from shapely.ops import transform

def make_buffer(coordinates):

    aeqd_crs = CRS.from_proj4(f"+proj=aeqd +lat_0={lat} +lon_0={lon} +datum=WGS84")
    to_4326 = Transformer.from_crs(aeqd_crs, 4326, always_xy=True).transform
    to_aeqd = Transformer.from_crs(4326, aeqd_crs, always_xy=True).transform

    buffer_center = transform(to_aeqd, coordinates)
    buffer_aeqd = buffer_center.buffer(100000)  # 100 km
    buffer_4326 = transform(to_4326, buffer_aeqd)
    
    return gpd.GeoDataFrame(data={'geometry' : [buffer_4326]}, crs='EPSG:4326')

    

In [15]:
TEST_COORDS = Point(-107.9, 37.2)

buffer = make_buffer(TEST_COORDS)
forecastOffices = gdf_clean.set_crs('EPSG:4326')
points_within_buffer = gpd.sjoin(forecastOffices, buffer, how='inner', predicate='within')
points_within_buffer

,geometry,grid_x,grid_y,grid_id,original_point,index_right
0,POINT (-108.46940 36.75552),88,9,GJT,"(37.0, -108.5)",0
1,POINT (-108.04584 36.87517),104,8,GJT,"(37.0, -108.05555555555556)",0
2,POINT (-107.59331 37.07489),71,209,ABQ,"(37.0, -107.61111111111111)",0
3,POINT (-107.00029 36.95378),87,208,ABQ,"(37.0, -107.16666666666667)",0
10,POINT (-108.49982 37.47399),90,29,GJT,"(37.44444444444444, -108.5)",0
11,POINT (-108.29386 37.34660),106,28,GJT,"(37.44444444444444, -108.05555555555556)",0
12,POINT (-107.59481 37.23525),121,26,GJT,"(37.44444444444444, -107.61111111111111)",0
13,POINT (-107.16843 37.44179),137,25,GJT,"(37.44444444444444, -107.16666666666667)",0
21,POINT (-108.00119 37.99462),107,47,GJT,"(37.888888888888886, -108.05555555555556)",0
22,POINT (-107.66464 37.81109),123,46,GJT,"(37.888888888888886, -107.61111111111111)",0


In [16]:
import statistics


DESIRED_WEATHER = 70

points_within_buffer['mean_temp'] = np.nan
points_within_buffer['desired_temp_diff'] = np.nan

for idx, row in points_within_buffer.iterrows():
    print("Requesting", row['grid_x'], row['grid_y'], row['grid_id'])
    response_raw = requests.get(f"https://api.weather.gov/gridpoints/{row['grid_id']}/{row['grid_x']},{row['grid_y']}/forecast/hourly")
    if response_raw.status_code == 200:
        response = response_raw.json()
        periods = response['properties']['periods']
        ave_temps = statistics.mean([period['temperature'] for period in periods if 1 <= period['number'] <= 48])
        points_within_buffer.at[idx, 'mean_temp'] = ave_temps
        points_within_buffer.at[idx, 'desired_temp_diff'] = abs(ave_temps - DESIRED_WEATHER)

destinations_sorted = points_within_buffer.sort_values('desired_temp_diff')      
destinations_sorted 
    

Requesting 88 9 GJT
Requesting 104 8 GJT
Requesting 71 209 ABQ
Requesting 87 208 ABQ
Requesting 90 29 GJT
Requesting 106 28 GJT
Requesting 121 26 GJT
Requesting 137 25 GJT
Requesting 107 47 GJT
Requesting 123 46 GJT


,geometry,grid_x,grid_y,grid_id,original_point,index_right,mean_temp,desired_temp_diff
0,POINT (-108.46940 36.75552),88,9,GJT,"(37.0, -108.5)",0,56.708333,13.291667
1,POINT (-108.04584 36.87517),104,8,GJT,"(37.0, -108.05555555555556)",0,54.854167,15.145833
2,POINT (-107.59331 37.07489),71,209,ABQ,"(37.0, -107.61111111111111)",0,53.937500,16.062500
3,POINT (-107.00029 36.95378),87,208,ABQ,"(37.0, -107.16666666666667)",0,51.895833,18.104167
10,POINT (-108.49982 37.47399),90,29,GJT,"(37.44444444444444, -108.5)",0,50.958333,19.041667
13,POINT (-107.16843 37.44179),137,25,GJT,"(37.44444444444444, -107.16666666666667)",0,43.666667,26.333333
21,POINT (-108.00119 37.99462),107,47,GJT,"(37.888888888888886, -108.05555555555556)",0,41.354167,28.645833
12,POINT (-107.59481 37.23525),121,26,GJT,"(37.44444444444444, -107.61111111111111)",0,39.354167,30.645833
11,POINT (-108.29386 37.34660),106,28,GJT,"(37.44444444444444, -108.05555555555556)",0,38.770833,31.229167
22,POINT (-107.66464 37.81109),123,46,GJT,"(37.888888888888886, -107.61111111111111)",0,26.520833,43.479167


In [34]:
import mapclassify
import folium

classifier = mapclassify.NaturalBreaks(points_within_buffer['mean_temp'], k=5)
points_within_buffer['temperature_class'] = classifier.yb  # Class index (0 to k-1)

colors = ['darkblue', 'blue', 'lightblue', 'orange', 'red']  # 5 classes = 5 colors

m2 = folium.Map(location=[points_within_buffer.geometry.y.mean(), 
                          points_within_buffer.geometry.x.mean()],
                zoom_start=6)

# Step 4: Loop through each point and assign a marker based on class
for idx, row in points_within_buffer.iterrows():
    color = colors[row['temperature_class']]
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=f"Mean Temp: {row['mean_temp']:.1f}°F",
        icon=folium.Icon(color=color, icon="cloud")
    ).add_to(m2)

m2


In [35]:
points_within_buffer

,geometry,grid_x,grid_y,grid_id,original_point,index_right,mean_temp,desired_temp_diff,temp_class,temperature_class
0,POINT (-108.46940 36.75552),88,9,GJT,"(37.0, -108.5)",0,56.708333,13.291667,4,4
1,POINT (-108.04584 36.87517),104,8,GJT,"(37.0, -108.05555555555556)",0,54.854167,15.145833,4,4
2,POINT (-107.59331 37.07489),71,209,ABQ,"(37.0, -107.61111111111111)",0,53.937500,16.062500,4,4
3,POINT (-107.00029 36.95378),87,208,ABQ,"(37.0, -107.16666666666667)",0,51.895833,18.104167,3,3
10,POINT (-108.49982 37.47399),90,29,GJT,"(37.44444444444444, -108.5)",0,50.958333,19.041667,3,3
11,POINT (-108.29386 37.34660),106,28,GJT,"(37.44444444444444, -108.05555555555556)",0,38.770833,31.229167,1,1
12,POINT (-107.59481 37.23525),121,26,GJT,"(37.44444444444444, -107.61111111111111)",0,39.354167,30.645833,1,1
13,POINT (-107.16843 37.44179),137,25,GJT,"(37.44444444444444, -107.16666666666667)",0,43.666667,26.333333,2,2
21,POINT (-108.00119 37.99462),107,47,GJT,"(37.888888888888886, -108.05555555555556)",0,41.354167,28.645833,2,2
22,POINT (-107.66464 37.81109),123,46,GJT,"(37.888888888888886, -107.61111111111111)",0,26.520833,43.479167,0,0
